## Import all requiered libraries

In [ ]:
# basic librareis
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision.datasets import ImageFolder
# plotting and visualizations
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
import missingno as msno
# preprocessing
from keras.preprocessing.image import (ImageDataGenerator, 
                                       img_to_array, 
                                       array_to_img, 
                                       load_img)

from sklearn.model_selection import train_test_split
# metrics
from sklearn.metrics import (confusion_matrix, 
                             classification_report, 
                             accuracy_score, 
                             f1_score, 
                             roc_auc_score,
                             plot_confusion_matrix)
# modeling
import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten,BatchNormalization,Activation,Input

from keras.applications import resnet50
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
#from keras.applications.vgg16 import VGG16



from keras import backend as K
K.clear_session()

# model plotting
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

# mesc
from sklearn.utils import shuffle


print('!Done')

## Data exploration

In [ ]:
train_dir = '../input/100-bird-species/train'
test_dir = '../input/100-bird-species/test'
validation_dir = '../input/100-bird-species/valid'
print('No. of Train Classes:' + str(len(os.listdir(train_dir))))
print('No. of Test Classes:' + str(len(os.listdir(test_dir))))
print('No. of Validation Classes:' + str(len(os.listdir(validation_dir))))
print("****************************")

train_data = ImageFolder(train_dir)
test_data = ImageFolder(test_dir)
validation_data = ImageFolder(validation_dir)

print('No. of Train Images:' + str(len(train_data)))
print('No. of Test Images:' + str(len(test_data)))
print('No. of Validation Images:' + str(len(validation_data)))


In [ ]:
#Classes
classes = list(train_data.class_to_idx.keys())
classes

*The dataset has 200 bird species meaning that there are 200 different classes or outputs we would like to predict with the trained model. The cell lists the classes from the train dataset. The same set of classes are present in the validation and test datasets.*

In [ ]:
from collections import Counter
img_counter = Counter([train_data.classes[labels] for _,labels in train_data])
img_counter

In [ ]:
train_data_count = np.array(list(img_counter.items())) #[:,0]
train_data_count

In [ ]:
img_counter = Counter([test_data.classes[labels] for _,labels in test_data])
img_counter

In [ ]:
img_counter = Counter([validation_data.classes[labels] for _,labels in validation_data])
img_counter

*We present also the number of images per species of birds in each dataset. For the validation and test datasets, there are 5 images for each class while for the training dataset, the number of images may be different according the class considered.*

 *Let’s check the characteristic of a single image. As we can see, the image has size 224x224 with three channels (RGB).*

In [ ]:
train_data[0]

*Now we display few images of the bird from dataset*

In [ ]:
image, label = train_data[0]
plt.imshow(image)
print("Numeric label: " + str(label))
print("Textual label: " + str(classes[label]))

### Image agumentation data preparation with ImageDataGenerator

In [ ]:
#First Read this: https://keras.io/api/preprocessing/image/#flow_from_directory
#https://keras.io/api/preprocessing/image/
size  = 224
batch_size = 256

*Create tf.data.Dataset for training, validation and test using Keras image_dataset_from_directory*

In [ ]:
train_datagen = ImageDataGenerator(
                             #rotation_range=10,
                             #zoom_range=0.1,
                             #horizontal_flip=True,
                             #fill_mode='nearest',
                             #width_shift_range=0.1,
                             #height_shift_range=0.1,
                             #preprocessing_function = preprocess_input,
                             rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(size, size), 
        #color_mode='rgb',
        batch_size=batch_size,
        class_mode='sparse')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
    #preprocessing_function = preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,  
        target_size=(size, size), 
        #color_mode='rgb',
        batch_size=batch_size,
        class_mode='sparse')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
    #preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(
        test_dir,  
        target_size=(size, size),  
        batch_size=batch_size,
        #color_mode='rgb',
        class_mode='sparse',
        shuffle=False)  #Whether to shuffle the data. Default: True. If set to False, sorts the data in alphanumeric order.
        #Set this to False, because you need to yield the images in “order”, to predict the outputs and match them with their unique ids or filenames.

## Classes of Birds

In [ ]:
train_generator.class_indices

# Transfer Learning Using RESNET_101

In [ ]:
from keras.applications import ResNet101V2
convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False

In [ ]:
model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(275,activation='softmax'))
print(model.summary())

In [ ]:
opt=tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)

In [ ]:
history=model.fit(train_generator,validation_data=validation_generator,
         epochs=5)

## Accuracy Plot

In [ ]:
plt.plot(history.history['accuracy'], c = 'red')
plt.plot(history.history['val_accuracy'], c = 'green')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc = 'lower right')


## Loss Plot

In [ ]:
plt.plot(history.history['loss'],c='red')
plt.plot(history.history['val_loss'],c='green')
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='upper right')

In [ ]:
model.save("model_ResNet101_BirdClassification.h5")

## Test Accuracy

In [ ]:
model.evaluate(test_generator)

## Prediction

In [ ]:
#from keras.models import load_model
#modelt = load_model("./model_ResNet101_BirdClassification.h5")


In [ ]:
#Function for predicting bird class
dic=train_generator.class_indices
icd={k:v for v,k in dic.items()}
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=np.argmax(model.predict(img), axis=-1)
    probability=round(np.max(model.predict(img)*100),2)
    #print ('Bird Is',icd[answer[0]], 'With probability',probability)
    print (probability, ' % chances are there that the Bird Is',icd[answer[0]])



### ALBATROSS

In [ ]:
img= '../input/100-bird-species/test/ALBATROSS/2.jpg'
pic=load_img(img,target_size=(224,224,3))
plt.imshow(pic)
output(img)

### BALD EAGLE

In [ ]:
img= '../input/100-bird-species/test/BALD EAGLE/2.jpg'
pic=load_img(img,target_size=(224,224,3))
plt.imshow(pic)
output(img)

### BIRD OF PARADISE

In [ ]:
img= '../input/100-bird-species/test/BIRD OF PARADISE/4.jpg'
pic=load_img(img,target_size=(224,224,3))
plt.imshow(pic)
output(img)

## Predict

In [ ]:
y_real = test_generator.classes
print(y_real)

In [ ]:
predictions = model.predict(test_generator)

y_pred = np.argmax(predictions, axis=-1)

In [ ]:
y_pred

## Confusion Matrix

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_real.reshape(len(y_real),1)),1))

In [ ]:
cm=confusion_matrix(y_real, y_pred)
cm

In [ ]:
#A lot classes fot plot it :)
#from mlxtend.plotting import plot_confusion_matrix
#plot_confusion_matrix(conf_mat=cm, figsize=(9,9), class_names = classes, show_normed=False)
#plt.tight_layout()

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_real,y_pred, digits = 4))